# Preprocesar encabezados

Este script remplaza dobles espacios, saltos de linea y dobles saltos de linea en los encabezados.

Es necesario crear un archivo llamado `mongo_uri.txt`en el mismo directorio de este script. Este archivo deberá contener la uri de conexión al replica set de MongoDB Atlas

In [1]:
from pymongo import MongoClient, UpdateOne
import re
from pprint import pprint

In [2]:
urifile = open("mongo_uri.txt", 'r', encoding='utf-8')
uri = urifile.read()
client = MongoClient(uri)
db = client.boletines_db
pprint(db.boletines.find_one())

{'_id': ObjectId('5ec3089dd380e649eb619ce3'),
 'boletin': '+----+--+--+ | [] | | | +----+--+--+\n'
            'Juntos haremos historia\n'
            'BOLETÍN CAMPAÑA-002\n'
            'ASEGURA AMLO QUE EN CAMPAÑA TODO SERÁ AMOR Y PAZ, QUE LOS OTROS '
            'CANDIDATOS SE AHORREN SUS PROVOCACIONES\n'
            '- Adelanta López Obrador adelantó que no pasará nada el primero '
            'de julio cuando se lleven a cabo las elecciones presidenciales\n'
            '- Expresa esperará que Donald Trump se serene y lo hará entrar en '
            'razón\n'
            '- Invita a que se realice un encuentro público entre técnicos '
            'para debatir si es viable o no la construcción del nuevo '
            'aeropuerto en el Lago de Texcoco\n'
            'SANTA CATARINA, NUEVO LEÓN, 02 DE ABRIL DE 2018.- Andrés Manuel '
            'López Obrador aseguró que en campaña todo será amor y paz, “que '
            'se ahorren cualquier provocación los candidatos de los otros

In [3]:
remplazos = [
    # Remplazo dobles lineas por un identificador especial
    (r'\n\n', '<dnl>'),
    # Remplazo lineas simples por espacios
    (r'\n', ' '),
    # Remplazo identificador de linea doble por linea simple
    (r'<dnl>', '\n'),
    # Remplazo caracter &nbsp (\u00A0) por espacio
    (r'\u00A0', ' '),
    # Remplazo tabulador por espacio
    (r'\t', ' '),
    # Remplazo repeticiones de más de dos espacios por espacio simple
    (r' ( )+', ' '),
]

In [5]:
update_ops = []
for doc in db.boletines.find({}, {"encabezado":1}):
    filtro = { '_id' : doc['_id']}
    for patron, remplazo in remplazos:
        doc['encabezado'] = re.sub(patron, remplazo, doc['encabezado'])
    update = {'$set': {'encabezado':doc['encabezado']}}
    update_ops.append(UpdateOne(filtro, update))
resultado = db.boletines.bulk_write(update_ops)
print(resultado.bulk_api_result)

{'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 344, 'nModified': 0, 'nRemoved': 0, 'upserted': []}


In [3]:
client.close()